In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp/scripts_python'

In [3]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import cv2

In [5]:
def safe_mkdir(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

In [6]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [53]:
def save_video(videodata,ofn='outvideo.mp4'):
    # 20FPS, opens in quicktime
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-pix_fmt' : 'yuv420p','-codec': 'h264', '-filter:v' : 'fps=20'})
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=20'})
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8))

In [ ]:
def return_frame2(img,enlarge=0):
    
    trained_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #img = cv2.imread(image_fn)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cordinates = trained_data.detectMultiScale(gray_img)
    face_cordinates = face_cordinates + [-enlarge,-enlarge,enlarge*2,enlarge*2]
    
    return face_cordinates

def crop_video_to_rect2(video_data,x,y,h,w):
    return np.array([crop_frame_to_rect2(video_data[i,:,:,:],x, y, w, h) for i in range(video_data.shape[0])])

def crop_frame_to_rect2(img,x,y,h,w):
    return img[np.arange(y,y+h),:,:][:,np.arange(x,x+w),:]

def resize_video(video_data,vid_size_target=(512,512)):
    from PIL import Image, ImageEnhance,ImageStat
    import numpy as np
    
    #vid_size_target = np.array((512,512))
    vid_size_target = np.array(vid_size_target)
    vid_size = np.array((video_data.shape[1],video_data.shape[2]))
    nframes = video_data.shape[0]
    nchannels = video_data.shape[-1]
    video_data_new = np.zeros((nframes,vid_size_target[0],vid_size_target[1],nchannels))
    for f in range(nframes):
        video_data_new[f,:,:,:] = np.array(Image.fromarray(video_data[f,:,:,:]).resize(vid_size_target[-1::-1],Image.ANTIALIAS))
        
    return video_data_new

In [39]:
indir = '../disfa/Video_RightCamera_cut/'
files = [os.path.join(indir,file) for file in os.listdir(indir) if file.endswith('.mp4')]
files.sort()
files[0:5]

['../disfa/Video_RightCamera_cut/RightVideoSN001_Comp_disgust_1.mp4',
 '../disfa/Video_RightCamera_cut/RightVideoSN001_Comp_fear_1.mp4',
 '../disfa/Video_RightCamera_cut/RightVideoSN001_Comp_happiness_1.mp4',
 '../disfa/Video_RightCamera_cut/RightVideoSN001_Comp_happiness_2.mp4',
 '../disfa/Video_RightCamera_cut/RightVideoSN001_Comp_sadness_1.mp4']

In [9]:
outdir = '../disfa/Video_RightCamera_cut_cropped/'
safe_mkdir(outdir)

In [56]:
for vid_fn in tqdm(files): 
    ofn = os.path.join(outdir,vid_fn.split('/')[-1])
    if not os.path.exists(ofn):
        
        video_data = load_video(vid_fn)

        frame_rects = np.array([return_frame2(video_data[i,:,:,:],enlarge=0)[0] for i in range(video_data.shape[0])])
        mean_frame = frame_rects.mean(axis=0).astype(int)
        x, y, w, h = mean_frame
        video_data_cropped = crop_video_to_rect2(video_data,x,y,h,w)
        video_data_cropped = resize_video(video_data_cropped,vid_size_target=(512,512))
        
        save_video(video_data_cropped,ofn)

  6%|██▎                                     | 11/189 [04:53<1:19:11, 26.69s/it]


KeyboardInterrupt: 